In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
from scipy import stats
import numpy as np

In [ ]:
from autumn.tools.runs import ManagedRun

from autumn.tools.runs.calibration.utils import get_posteriors
from autumn.tools.utils.pandas import pdfilt

In [ ]:
#run_id = "covid_19/victoria/1630409372/1cbdde5"
#run_id = "covid_19/malaysia/1630499991/66b9a81"
run_id = "covid_19/victoria/1630616144/d8164de"

In [ ]:
mr = ManagedRun(run_id)

In [ ]:
mcmc_runs = mr.calibration.get_mcmc_runs()
mcmc_params = mr.calibration.get_mcmc_params()

In [ ]:
# Burn 500 runs (from every chain), and filter by accept only
#filt_idx = pdfilt(mcmc_runs, ['accept==1','run >= 500']).index

In [ ]:
# These are the params filtered by the above
#pset = mcmc_params.loc[filt_idx]

In [ ]:
parameters = list(pset.columns)

In [ ]:
pset = get_posteriors(mcmc_params, mcmc_runs, 500)

In [ ]:
def fit(data, dist_name, plot=True):
    dist = getattr(stats.distributions, dist_name)
    dist_params = dist.fit(data)
    xrange = np.linspace(data.min(),data.max(), 200)
    
    if plot:
        data.hist(density=True,bins=50)
        pdf = pd.Series([(dist.pdf(x, *dist_params)) for x in xrange], index=xrange)
        pdf.plot()
    
    return dist_params

In [ ]:
parameter = pset.columns[10]
data = pset[parameter]
print(parameter)

In [ ]:
fit(data, 'norm')

In [ ]:
fit(data, 'beta')

In [ ]:
fit(data,'uniform')